In [1]:
!pip -V

pip 20.2.4 from /opt/anaconda3/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!pip install investpy

In [6]:
import investpy
import pandas
from datetime import datetime

countriesAvailable = investpy.get_certificate_countries()
print(countriesAvailable)

search_result = investpy.search_quotes(text='apple', products=['stocks'],
                                       countries=['united states'], n_results=1)
print(search_result)

default_currency = search_result.retrieve_currency()
print(default_currency)

['france', 'germany', 'italy', 'netherlands', 'sweden']
{"id_": 6408, "name": "Apple Inc", "symbol": "AAPL", "country": "united states", "tag": "/equities/apple-computer-inc", "pair_type": "stocks", "exchange": "NASDAQ"}
USD


In [88]:
import requests
from bs4 import BeautifulSoup as bs

def make_usd_index():
    main_url = "https://kr.investing.com/currencies/us-dollar-index"
    response = requests.get( main_url, headers={"User-agent": "Mozilla/5.0"} )
    soup = bs(response.text, 'html.parser')

    historical_usd_idx = soup.find("div",{"class":"clear overviewDataTable overviewDataTableWithTooltip"})
    idx = 0
    dict_usd_idx = {}
    for row in historical_usd_idx:
        row_string = str(row).strip()
        if len(row_string) == 0: continue
        idx += 1
        temp = str(row_string.split('<span class="float_lang_base_1">')[1:]).replace("['","")
        list_temp = temp.replace("</span><span class=","").split('"float_lang_base_2 bold">')
        dict_usd_idx[idx] = [list_temp[0].replace(" ",""), list_temp[1].replace("</span></div>']","")]
    
    return dict_usd_idx

def make_currency():
    main_url = "https://kr.investing.com/currencies/usd-krw"
    response = requests.get( main_url, headers={"User-agent": "Mozilla/5.0"} )
    soup = bs(response.text, 'html.parser')

    list_usd_krw = soup.find("div",{"class":"main-current-data"})
    idx = 0
    dict_usd_krw = {}
    for row in list_usd_krw:
        row_string = str(row).strip()
        if len(row_string) == 0: continue
        idx += 1
        if idx == 1:
            dict_usd_krw[idx] = row_string.split('<div class="')[1].split(" ")[0]
        else:
            list_val = []
            for val in row_string.split("\n"):
                if "span class" not in val: continue
                list_val.append(val.split('">')[1].replace("</span>",""))
            dict_usd_krw[idx] = list_val
            
    list_usd_krw = soup.find("div",{"class":"clear overviewDataTable overviewDataTableWithTooltip"})
    for row in list_usd_krw:
        row_string = str(row).strip()
        if len(row_string) == 0: continue
        idx += 1
        temp = str(row_string.split('<span class="float_lang_base_1">')[1:]).replace("['","")
        list_temp = temp.replace("</span><span class=","").split('"float_lang_base_2 bold">')
        dict_usd_krw[idx] = [list_temp[0].replace(" ",""), list_temp[1].replace("</span></div>']","")]
    
    return dict_usd_krw
    
dict_usd_idx = make_usd_index()
dict_usd_krw = make_currency()

dict_usd_krw

{1: 'downArrow',
 2: ['1,177.66', '-0.12', '-0.01%'],
 3: ['전일종가', '1,177.78'],
 4: ['매수', '1,177.48'],
 5: ['금일변동', '1,173.91 - 1,181.7'],
 6: ['금일시가', '1,177.7'],
 7: ['매도', '1,177.83'],
 8: ['52주변동폭', '1,080.02 - 1,200.9'],
 9: ['1년변동률', '4.41%']}

In [85]:
# USD Index 52w Average
year_avg_idx = (float(dict_usd_idx[10][1].split(" - ")[0].replace(",","")) + float(dict_usd_idx[10][1].split(" - ")[1].replace(",",""))) / 2
year_avg_idx

91.8675

In [86]:
# Now Index
cur_idx = float(dict_usd_idx[4][1].replace(",",""))
cur_idx

93.738

In [87]:
# USD KRW 52w Average
year_avg_cur = (float(dict_usd_krw[8][1].split(" - ")[0].replace(",","")) + float(dict_usd_krw[8][1].split(" - ")[1].replace(",",""))) / 2
year_avg_cur

1140.46

In [90]:
# Now Currency
now_cur = float(dict_usd_krw[2][0].replace(",",""))
now_cur

1177.66

In [92]:
# Dollar Gap Rate
usd_gap_rate = cur_idx / now_cur * 100
usd_gap_rate

7.959682760728902

In [93]:
# Dollar Gap Rate - 52w
year_avg_usd_gap_rate = year_avg_idx / year_avg_cur * 100
year_avg_usd_gap_rate

8.055302246461954

In [96]:
# Proper USD KRW
proper_cur = round(cur_idx / year_avg_usd_gap_rate * 100, 2)
proper_cur

1163.68

In [102]:
if now_cur < proper_cur * 0.99:
    print("Buy")
elif now_cur > proper_cur * 1.1:
    print("Sell")
else:
    print(now_cur, proper_cur * 0.99, proper_cur * 1.1)
    print("Wait...")

1177.66 1152.0432 1280.0480000000002
Wait...


In [99]:
proper_cur * 0.99

1152.0432

In [103]:
for key, val in dict_usd_idx.items():
    print(key, val)

1 ['전일종가', '93.758']
2 ['만기월', '2021년 12월']
3 ['틱크기', '0.01']
4 ['금일시가', '93.738']
5 ['계약단위', '1,000 x 물가지수']
6 ['틱가치', '10']
7 ['금일변동', '93.523 - 93.782']
8 ['결제방식', '실물인수도']
9 ['티커', 'DX']
10 ['52주변동폭', '89.165 - 94.57']
11 ['결제일', '2021/12/13']
12 ['포인트가치', '1 = $1000']
13 ['1년변동률', '0.89%']
14 ['최종롤오버일', '2021/09/12']
15 ['개월', 'HMUZ']
